In [1]:
import sys
sys.path.insert(0, '../src')
from bird_dataset import *

In [2]:
import os
import torch
import pandas as pd
# from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import PIL.Image as Image
import torch.nn as nn
import torchvision
import torch.optim as optim
import sys
from captum.attr import GuidedGradCam
# import cv2
from PIL import Image
from matplotlib.colors import LinearSegmentedColormap
from captum.attr import visualization as viz

# sys.path.insert(0, '../src')
from bird_dataset import *
# from XAI_birds_dataloader import *
from tqdm import tqdm
from models.multi_task_model import *
# from XAI_birds_dataloader import *
from XAI_BirdAttribute_dataloader import *
import pickle

from download import *

import shutil

In [3]:
bd = BirdDataset(data_dir='../CUB_200_2011/', species_file='classes-families.txt', save=True, save_file='images-families.pkl', preload_dir='../processed_data/')

In [ ]:
bd.fam[bd.bird_to_fam[bd.images[100]['filepath'].split('/')[0]]]

In [ ]:
bird_names = pd.Series([i.split('_')[-1] for i in os.listdir('../CUB_200_2011/images')]).value_counts()
bird_families = bird_names[bird_names > 1]
bird_fam_dict = {i: i.split('_')[-1] for i in os.listdir('../CUB_200_2011/images')}
fam_dict = dict(zip(bird_families.index, range(len(bird_families))))

In [ ]:
species_file = [str(idx)+' '+str(j)+'\n' for idx, j in enumerate([i for i in os.listdir('../CUB_200_2011/images') if bird_fam_dict[i] in bird_families.index])]

In [ ]:
bird_names

In [ ]:
name = 'families'
with open(f'../CUB_200_2011/classes-{name}.txt', 'w') as f:
    for spec in species_file:
        f.write(spec)

In [ ]:
len(bird_families)

In [ ]:
# bird_names.reset_index()[pd.Series(bird_names.index).apply(lambda x: x[0].isalpha()==False)]

In [ ]:
BirdDataset?

In [ ]:
bd = BirdDataset(save=True, data_dir='../CUB_200_2011/', preload_dir='../processed_data/')

In [ ]:
# class_labels = [bd.images[i]['class_label'] for i in bd.images]

In [3]:
bd = BirdDataset(preload=True, data_dir='../CUB_200_2011/', species_file='classes-families.txt', preload_dir='../processed_data/', preload_file='images-families.pkl')

In [6]:
# bd.images

In [7]:
torch.__version__

'1.6.0+cu101'

In [ ]:
# tasks: ["has_bill_shape","has_wing_shape", "has_breast_pattern"]

In [4]:
# bd = BirdDataset(preload=True, attr_file='attributes')
vgg16 = models.vgg16_bn(pretrained=True)
trans = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.ToTensor()
])

full_dataset = Bird_Attribute_Loader(bd, attrs=None, verbose=False, species=True, transform=trans)
np.random.seed(42)
train_indices = np.random.choice(a=range(len(full_dataset)), size=int(0.8*len(full_dataset)), replace=False)
val_indices = list(set(range(len(full_dataset))) - set(train_indices))
train_bird_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_bird_dataset = torch.utils.data.Subset(full_dataset, val_indices)
# train_bird_dataset = Bird_Attribute_Loader(bd, attrs=None, verbose=True, species=True, transform=trans, train=True)
# val_bird_dataset = Bird_Attribute_Loader(bd, attrs=None, verbose=True, species=True, transform=trans, train=False, val=True)
model = MultiTaskModel(vgg16, full_dataset)
loss_func = MultiTaskLossWrapper(full_dataset)

In [6]:
# model

In [ ]:
for i in tqdm(full_dataset):
    pass
#     print(i['labels'])

 49%|████▉     | 5810/11780 [00:42<00:47, 125.11it/s]

In [14]:
torch.LongTensor([val_bird_dataset[0]['labels'] , val_bird_dataset[1]['labels']])

tensor([[  5,   3,   3,   0],
        [  6,   3,   3, 169]])

In [23]:
# def _filter_images_by_attributes():
#         filt_images = []
#         attr_indices = []
#         for i, img in enumerate(full_dataset.images):
#             check=0
#             attr_index = []
#             attrs = []
            
#             for idx, attr in enumerate(img['attributes']):
#                 if isinstance(full_dataset.attrs, str):
#                     if full_dataset.attrs in attr:
#                         filt_images.append(img)
#                         attr_indices.append(idx)
#                         break
#                 elif isinstance(full_dataset.attrs, list):
#                     for attribute in full_dataset.attrs:
#                         if attribute in attr:
#                             check+=1
#                             attr_index.append(idx)
#                             attrs.append(attr)
#                 else: raise(ValueError, "self.attrs must be a string or a list of strings")
#             if check==len(full_dataset.attrs): # only append to images/indices if all attributes in self.attrs are in the images attributes
#                 unique_attrs = [attr.split('::')[0] for attr in attrs]
#                 if len(set(unique_attrs)) == len(full_dataset.attrs):
#                     img['attrs'] = attrs
#                     filt_images.append(img)
#                     attr_indices.append((list(attr_index)))
# #                 else: print("Attributes aren't unique:",attr)
#         return filt_images, attr_indices

In [24]:
train_bird_dataset[0]

{'image': tensor([[[0.5569, 0.5451, 0.5216,  ..., 0.6353, 0.5961, 0.5765],
          [0.5451, 0.5294, 0.4980,  ..., 0.6431, 0.5922, 0.5569],
          [0.5216, 0.5137, 0.4549,  ..., 0.6392, 0.5843, 0.5412],
          ...,
          [0.0314, 0.0196, 0.0157,  ..., 0.3882, 0.4039, 0.4118],
          [0.0235, 0.0196, 0.0157,  ..., 0.3843, 0.4000, 0.4118],
          [0.0157, 0.0157, 0.0196,  ..., 0.3922, 0.4078, 0.4157]],
 
         [[0.5098, 0.4902, 0.4824,  ..., 0.6510, 0.6118, 0.5882],
          [0.4980, 0.4941, 0.4588,  ..., 0.6588, 0.6157, 0.5804],
          [0.4784, 0.4824, 0.4275,  ..., 0.6588, 0.6078, 0.5608],
          ...,
          [0.0392, 0.0235, 0.0196,  ..., 0.3882, 0.4000, 0.4157],
          [0.0353, 0.0235, 0.0196,  ..., 0.3882, 0.4000, 0.4157],
          [0.0275, 0.0235, 0.0235,  ..., 0.3882, 0.4000, 0.4157]],
 
         [[0.4431, 0.4588, 0.4706,  ..., 0.4824, 0.4353, 0.4078],
          [0.4510, 0.4706, 0.4588,  ..., 0.4824, 0.4392, 0.4078],
          [0.4549, 0.4667, 0.41

In [25]:
# sum(['has_wing_shape' in full_dataset.bd.images[key]['attributes'][i] for key in full_dataset.bd.images.keys() for i in range(len(full_dataset.bd.images[key]['attributes']))])

In [26]:
# len(full_dataset.bd.images)

In [21]:
valloader = DataLoader(val_bird_dataset, batch_size=4)

In [26]:
with torch.no_grad():
    num_correct=0
    val_losses = []
    data_iter = iter(valloader)
    for val_data in tqdm(data_iter):
        val_inputs, val_labels = val_data['image'].cuda(), torch.LongTensor(val_data['labels']).cuda()
        val_outputs = model(val_inputs)
#         opt.zero_grad() # zero the parameter gradients
        val_predicted = [torch.max(i, 1)[1] for i in val_outputs]
        num_correct += sum(np.array(val_labels.cpu()).flatten()==np.array(val_predicted[0].cpu()).flatten())
        print(val_labels)
        print(val_predicted)
        print(num_correct)        
        break
        val_losses.append(loss_func(val_outputs, val_labels).item())


  0%|          | 0/589 [00:00<?, ?it/s]

tensor([  0, 139, 139,   0], device='cuda:0')
[tensor([ 18,  88, 139,  13], device='cuda:0')]
1


In [51]:
train_data = iter(DataLoader(train_bird_dataset, batch_size=4))

In [52]:
val_data = iter(DataLoader(val_bird_dataset))

In [53]:
print(val_data)

In [54]:
torch.utils.data.random_split?

In [55]:
train_size = int(0.8*len(full_dataset))
test_size = int(0.2*len(full_dataset))+1

In [63]:
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42) )

In [64]:
train_data = DataLoader(train_dataset, batch_size=4)

In [65]:
d = iter(train_data)

In [66]:
d

In [47]:
(val_data)

NameError: name 'val_data' is not defined

In [67]:
torch.utils.data.random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))

In [70]:
for data in train_data:
    x = data
    print(data)
    break

{'image': tensor([[[[0.8078, 0.8118, 0.8157,  ..., 0.4510, 0.4431, 0.4431],
          [0.8078, 0.8078, 0.8118,  ..., 0.4510, 0.4471, 0.4471],
          [0.8078, 0.8118, 0.8118,  ..., 0.4471, 0.4431, 0.4431],
          ...,
          [0.4706, 0.5725, 0.6157,  ..., 0.5373, 0.5373, 0.5373],
          [0.4471, 0.4510, 0.4588,  ..., 0.5373, 0.5373, 0.5373],
          [0.4549, 0.4510, 0.4510,  ..., 0.5451, 0.5451, 0.5451]],

         [[0.8706, 0.8745, 0.8784,  ..., 0.4980, 0.4902, 0.4902],
          [0.8706, 0.8706, 0.8706,  ..., 0.4980, 0.4941, 0.4941],
          [0.8706, 0.8745, 0.8745,  ..., 0.4941, 0.4902, 0.4902],
          ...,
          [0.5882, 0.6627, 0.6941,  ..., 0.5804, 0.5804, 0.5804],
          [0.5765, 0.5804, 0.5804,  ..., 0.5804, 0.5804, 0.5804],
          [0.5882, 0.5843, 0.5843,  ..., 0.5882, 0.5882, 0.5882]],

         [[0.4863, 0.4902, 0.4980,  ..., 0.2431, 0.2392, 0.2392],
          [0.4863, 0.4863, 0.4902,  ..., 0.2392, 0.2353, 0.2353],
          [0.4863, 0.4902, 0.494

In [73]:
torch.stack(x['labels'])

tensor([[  6,   6,   6,   7],
        [  3,   0,   0,   1],
        [  2,   0,   3,   1],
        [151,  59, 191, 119]])

In [69]:
# len([i['image_id'] for i in train_bird_dataset.images])# + len([i['image_id'] for i in val_bird_dataset.images])

In [11]:
len(((set([i['image_id'] for i in train_bird_dataset.dataset.images]).intersection(set([i['image_id'] for i in val_bird_dataset.dataset.images])))))

11780

In [12]:
# (set([i['image_id'] for i in train_bird_dataset.images])) - len((set([i['image_id'] for i in train_bird_dataset.images])-(set([i['image_id'] for i in val_bird_dataset.images]))))

In [13]:
# len(set([i['image_id'] for i in val_bird_dataset.images]))

# Multi-Task Training

In [5]:
from multi_task_training import *

In [6]:
mtt = MultiTaskTraining(model, train_bird_dataset, val_bird_dataset, loss_func, epochs=50, lr=0.0001, patience=7, batch_size=1)

cuda:0
species__family


In [ ]:
loss = sum([nn.CrossEntropyLoss()(preds[i].reshape(1, -1), labels[i].reshape(len(self.dataset.class_dict))) for i in range(len(preds))])

In [12]:
torch.Tensor([[63,  0]])[0].size()

torch.Size([2])

In [7]:
mtt.train()

0it [00:00, ?it/s]


LABELS: tensor([[127,  33]])
PREDICTIONS: (tensor([[ 1.1869,  0.2762,  0.0607,  0.0169,  0.3121,  0.5991,  1.4568,  0.0127,
         -1.2741, -0.9705,  0.8816, -0.4713, -0.2603, -0.8248, -0.3904,  1.0208,
         -2.0527, -0.2700,  0.1863, -0.8016,  0.5367,  1.0074,  1.1655, -0.0029,
          0.1828,  1.0523,  1.0362,  2.2917, -0.6803,  0.0062,  0.7945,  0.9544,
          0.7100, -0.0393, -0.5320, -0.4488,  0.6781, -1.3071,  1.9142, -0.3373,
          0.1754,  0.5911,  0.3595,  1.9886,  0.2337,  1.2409,  0.1150, -1.0193,
         -1.2040, -0.2438, -1.3085,  2.1871,  0.7250, -0.3577, -0.7577, -0.2432,
          1.0317, -0.1090,  1.0346, -0.5093,  0.0993,  0.8481, -1.2935,  1.0787,
          1.0066,  0.0242, -0.6231,  1.2199,  1.1260, -0.1601,  0.2833,  0.6249,
         -0.6106,  0.5875,  0.8174, -0.2295,  0.1111,  0.0543, -0.1930,  1.6079,
          1.0507, -0.4968,  1.0142, -0.3443,  0.9901, -0.3888, -0.0792,  1.3265,
          0.7907, -1.0578, -1.1052,  1.0357,  0.7055, -1.9441,  0.

IndexError: dimension specified as 0 but tensor has no dimensions

In [15]:
# preds = (torch.Tensor([[-0.1654,  1.7167,  0.3085, -0.4484, -0.1295, -0.2691, -0.1973, -0.4742,
#          -0.9800],
#         [-0.7576,  0.1325,  0.6803,  0.9214, -0.4653,  1.9751,  0.3790, -0.5589,
#           0.2431],
#         [-0.2137,  0.2937, -0.4808, -2.0652, -1.2837, -2.0810,  0.6016,  1.0635,
#           0.3386],
#         [ 0.8723, -1.7949,  1.4109, -1.4373, -1.7372,  0.5594, -1.3526,  0.9271,
#          -0.4259]]), 
#          torch.Tensor([[-1.2771,  0.3203, -0.9238,  0.2083],
#         [-1.1229, -2.0029,  1.6569,  0.4800],
#         [ 0.1119,  1.6074, -1.6294, -1.9116],
#         [-0.4751, -1.4349, -0.7779, -0.4444]]), 
#          torch.Tensor([[-1.9170,  1.2044, -1.2757,  0.5097],
#         [ 0.2896, -0.0633, -1.6042, -0.0619],
#         [-0.2362,  1.4970,  0.0553, -0.1821],
#         [ 2.7936,  0.2265, -3.3088,  1.1016]]))

# labels = torch.Tensor([[  6,   0,   2, 101],
#         [  6,   2,   0,  17],
#         [  1,   0,   3, 145],
#         [  7,   0,   3,  42]])

In [25]:
labels[0][1]

tensor(0.)

In [52]:
for i in range(len(labels)):
    for j in range(len(preds)):
        print('labels:',labels[i][j].dim())
        print('preds:', preds[j][i].reshape().dim())
        nn.CrossEntropyLoss()(labels[i][j], preds[j][i])
        print()
    print()

labels: 0


TypeError: reshape() missing 1 required positional arguments: "shape"

In [27]:
preds[1][0]

tensor([-1.2771,  0.3203, -0.9238,  0.2083])

In [11]:
mtt.train()

0it [00:00, ?it/s]

LABELS: tensor([[  6,   0,   2, 101],
        [  6,   2,   0,  17],
        [  1,   0,   3, 145],
        [  7,   0,   3,  42]])


0it [00:12, ?it/s]

PREDICTIONS: (tensor([[-0.1654,  1.7167,  0.3085, -0.4484, -0.1295, -0.2691, -0.1973, -0.4742,
         -0.9800],
        [-0.7576,  0.1325,  0.6803,  0.9214, -0.4653,  1.9751,  0.3790, -0.5589,
          0.2431],
        [-0.2137,  0.2937, -0.4808, -2.0652, -1.2837, -2.0810,  0.6016,  1.0635,
          0.3386],
        [ 0.8723, -1.7949,  1.4109, -1.4373, -1.7372,  0.5594, -1.3526,  0.9271,
         -0.4259]], grad_fn=<AddmmBackward>), tensor([[-1.2771,  0.3203, -0.9238,  0.2083],
        [-1.1229, -2.0029,  1.6569,  0.4800],
        [ 0.1119,  1.6074, -1.6294, -1.9116],
        [-0.4751, -1.4349, -0.7779, -0.4444]], grad_fn=<AddmmBackward>), tensor([[-1.9170,  1.2044, -1.2757,  0.5097],
        [ 0.2896, -0.0633, -1.6042, -0.0619],
        [-0.2362,  1.4970,  0.0553, -0.1821],
        [ 2.7936,  0.2265, -3.3088,  1.1016]], grad_fn=<AddmmBackward>), tensor([[ 7.4822e-01, -1.4717e-01,  5.6113e-01, -1.3976e+00,  2.5080e+00,
         -1.4112e+00,  1.1762e+00,  2.3842e+00,  1.8115e+00, -1

ValueError: Expected input batch_size (1) to match target batch_size (4).

In [28]:
from tqdm import tqdm

In [29]:
train_bird_dataset

In [30]:
# bd.images[1401]

In [31]:
# train_bird_dataset[478]

In [38]:
full_dataset[0]['image'].mode

'RGB'

In [41]:
bw_idx = []
for i in tqdm(range(len(full_dataset))):
    if full_dataset[i]['image'].mode != 'RGB':
        print(i)
        bw_idx.append(i)
#         print(train_bird_dataset[i])
#         break

  8%|▊         | 899/11788 [00:00<00:06, 1659.40it/s]

447


 13%|█▎        | 1510/11788 [00:01<00:08, 1181.56it/s]

1400


 32%|███▏      | 3750/11788 [00:03<00:07, 1096.70it/s]

...

 47%|████▋     | 5599/11788 [00:04<00:05, 1056.59it/s]

5392


 55%|█████▍    | 6458/11788 [00:05<00:05, 1035.06it/s]

6320


100%|██████████| 11788/11788 [00:10<00:00, 1087.08it/s]


In [42]:
bw_idx

[447, 1400, 3616, 3618, 3779, 5028, 5392, 6320]

In [ ]:
bw_ids = [448, 1401, 3617, 3619, 3780, 5029, 5393, 6321]

In [44]:
for idx in bw_idx:
    print(full_dataset[idx])

image_id:  448
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x333 at 0x7FDAA8E13990>, 'labels': 8}
image_id:  1401
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x375 at 0x7FDAA8F935D0>, 'labels': 24}
image_id:  3617
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x326 at 0x7FDAA8E13B50>, 'labels': 62}
image_id:  3619
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x398 at 0x7FDAA76838D0>, 'labels': 62}
image_id:  3780
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x281 at 0x7FDAA8E13990>, 'labels': 65}
image_id:  5029
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=485x500 at 0x7FDAA8F93810>, 'labels': 86}
image_id:  5393
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x500 at 0x7FDAA8E13810>, 'labels': 92}
image_id:  6321
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=484x500 at 0x7FDAA76837D0>, 'labels': 107}


In [24]:
for i in tqdm(range(len(full_dataset))):
    if full_dataset[i]['image'].size() != torch.Size([3, 224, 224]):
        print(i)

  0%|          | 0/11788 [00:00<?, ?it/s]


AttributeError: 'Bird_Attribute_Loader' object has no attribute 'transform'

In [34]:
b_w_indices = [1407, 1444, 1681]
for idx in b_w_indices:
    print(val_bird_dataset[idx]['image'].size())

image_id:  3617
torch.Size([1, 224, 224])
image_id:  3780
torch.Size([1, 224, 224])
image_id:  5029
torch.Size([1, 224, 224])


In [ ]:
bd.images

In [ ]:
train_bird_dataset

In [33]:
b_w_indices = [478, 1761, 5146, 5244, 5650, 7718]
for idx in b_w_indices:
    print(train_bird_dataset[idx]['image'].size())

image_id:  1401
torch.Size([1, 224, 224])
image_id:  3780
torch.Size([1, 224, 224])
image_id:  448
torch.Size([1, 224, 224])
image_id:  5393
torch.Size([1, 224, 224])
image_id:  3617
torch.Size([1, 224, 224])
image_id:  3619
torch.Size([1, 224, 224])


In [14]:
for i, data in tqdm(enumerate(train_bird_dataset)):
    if data['labels']>199:
        print(i)
        print(data)
        break

98it [00:07, 12.54it/s]


KeyboardInterrupt: 

In [44]:
for i, data in tqdm(enumerate(mtt.trainloader, 0)):
    if data['labels'][0]>199:
        print(i)
        print(data)
        break

592it [00:47, 12.37it/s]

592
{'image': tensor([[[[0.8157, 0.7216, 0.6078,  ..., 0.9294, 0.8392, 0.7020],
          [0.8275, 0.7294, 0.6157,  ..., 0.9373, 0.8431, 0.7098],
          [0.8353, 0.7412, 0.6314,  ..., 0.9412, 0.8471, 0.7059],
          ...,
          [0.3020, 0.3020, 0.2941,  ..., 0.3294, 0.4353, 0.5255],
          [0.2980, 0.2980, 0.2941,  ..., 0.3490, 0.4588, 0.5412],
          [0.2824, 0.2980, 0.3020,  ..., 0.3765, 0.4824, 0.5569]],

         [[0.6784, 0.6235, 0.5529,  ..., 0.8196, 0.7647, 0.6941],
          [0.6863, 0.6235, 0.5529,  ..., 0.8275, 0.7686, 0.7059],
          [0.6941, 0.6314, 0.5608,  ..., 0.8353, 0.7686, 0.7020],
          ...,
          [0.2627, 0.2627, 0.2627,  ..., 0.3255, 0.4549, 0.5765],
          [0.2588, 0.2627, 0.2627,  ..., 0.3373, 0.4627, 0.5765],
          [0.2471, 0.2627, 0.2706,  ..., 0.3608, 0.4824, 0.5922]],

         [[0.3529, 0.2431, 0.1333,  ..., 0.5255, 0.3412, 0.1216],
          [0.3725, 0.2510, 0.1373,  ..., 0.5333, 0.3333, 0.1176],
          [0.3882, 0.2667, 0

In [22]:
mtt.train_set.class_dict

{'species': {0: 'Black_footed_Albatross',
  1: 'Laysan_Albatross',
  2: 'Sooty_Albatross',
  3: 'Groove_billed_Ani',
  4: 'Crested_Auklet',
  5: 'Least_Auklet',
  6: 'Parakeet_Auklet',
  7: 'Rhinoceros_Auklet',
  8: 'Brewer_Blackbird',
  9: 'Red_winged_Blackbird',
  10: 'Rusty_Blackbird',
  11: 'Yellow_headed_Blackbird',
  12: 'Bobolink',
  13: 'Indigo_Bunting',
  14: 'Lazuli_Bunting',
  15: 'Painted_Bunting',
  16: 'Cardinal',
  17: 'Spotted_Catbird',
  18: 'Gray_Catbird',
  19: 'Yellow_breasted_Chat',
  20: 'Eastern_Towhee',
  21: 'Chuck_will_Widow',
  22: 'Brandt_Cormorant',
  23: 'Red_faced_Cormorant',
  24: 'Pelagic_Cormorant',
  25: 'Bronzed_Cowbird',
  26: 'Shiny_Cowbird',
  27: 'Brown_Creeper',
  28: 'American_Crow',
  29: 'Fish_Crow',
  30: 'Black_billed_Cuckoo',
  31: 'Mangrove_Cuckoo',
  32: 'Yellow_billed_Cuckoo',
  33: 'Gray_crowned_Rosy_Finch',
  34: 'Purple_Finch',
  35: 'Northern_Flicker',
  36: 'Acadian_Flycatcher',
  37: 'Great_Crested_Flycatcher',
  38: 'Least_Flycat